In [1]:
from rdflib.plugins.stores import sparqlstore
import rdflib
from rdflib import URIRef, Literal, Graph, Dataset

import serialization
import discourse

import html
import loader
import discourse
import pandas as pd
from rdflib import Graph, Namespace
from rdflib.namespace import RDF, RDFS
import io
import pydotplus
from IPython.display import display, Image
from rdflib.tools.rdf2dot import rdf2dot
from IPython.core.display import HTML

import uuid

from itertools import chain

def visualize(g):
    stream = io.StringIO()
    rdf2dot(g, stream, opts = {display})
    dg = pydotplus.graph_from_dot_data(stream.getvalue())
    png = dg.create_png()

    display(Image(png))

def t2rdflibg(triples): #triples to rdflib graph
    g = Graph()
    for t in triples:
        g.add(t)
    return g


def nan2None(value):
    if pd.isnull(value) :
        return None
    else:
        return value


def get_data_rows(filename):
    data_rows_df = pd.read_csv(filename)
    data_rows=[]
    for i,r in data_rows_df.iterrows():
        r_d = {k:nan2None(v) for k,v in dict(r).items()}

        data_rows.append(r_d)
    del data_rows_df
    return data_rows


jena = sparqlstore.SPARQLUpdateStore("http://localhost:3030/modelg/query",context_aware=True)
jena.open(("http://localhost:3030/modelg/query", "http://localhost:3030/modelg/update"))
#jena.open(("http://localhost:3030/models/query", "http://localhost:3030/models/update"))

# Create a Dataset
ds = Dataset(store=jena, default_union=True, default_graph_base="http://base.raw")
#ds = Dataset(store=jena, default_union=True)
# Define a named graph within the dataset, this will contain all nodes contained within the graph referenced, or none if it doesn't exist yet.
sg = ds.graph(URIRef("http://config"))
sg = Graph()
master_g = ds.graph(URIRef("http://master"))
discourse_g = ds.graph(URIRef("http://discourse"))



* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation
* Owlready2 * WARNING: ObjectProperty http://www.semanticweb.org/tomk/ontologies/2022/11/serialization#MappingMetaTarget belongs to more than one entity types: [owl.AnnotationProperty, owl.ObjectProperty, owl.topObjectProperty]; I'm trying to fix it...
* Owlready2 * Running HermiT...
    java -Xmx2000M -cp /usr/local/python/py36/lib/python3.6/site-packages/owlready2/hermit:/usr/local/python/py36/lib/python3.6/site-packages/owlready2/hermit/HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:////var/folders/rd/q3bhk6_n56x75m05pp2sq7dr0000gn/T/tmpxi4ekctu


Serialization.Meta [Serialization.SerializationMetaData]
Serialization.Mapping [Serialization.SerializationSpecs]
Serialization.BatchDefinition [Serialization.SerializationSpecs]
Serialization.SerializationSpecs [Serialization.SerializationMetaData]
Serialization.BatchNode [Serialization.DataLoad]
Serialization.DataLoad [Serialization.SerializationMetaData]
Serialization.SerializationMetaData [owl.Thing]
Serialization.DataPropertyMapping [Serialization.Mapping]
Serialization.MetaDataProperty [Serialization.Meta]
Serialization.EntityMapping [Serialization.Mapping]
Serialization.MetaClass [Serialization.Meta]
Serialization.JobNode [Serialization.DataLoad]
Serialization.MetaProperty [Serialization.Meta]
Serialization.PropertyMapping [Serialization.Mapping]
Serialization.RowNode [Serialization.DataLoad]
Serialization.Serialization [Serialization.SerializationSpecs]


* Owlready2 * HermiT took 0.487274169921875 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [2]:
def meta_data_package_template(field_d):
    dc_terms_base = "http://purl.org/dc/terms/"
    rdf_form = {}
    for k,v in field_d.items():
        rdf_form[URIRef(dc_terms_base + k)] = Literal(v)
    return rdf_form


def triples_to_quads(triples, graph="http://master"):
    for s,p,o, *_ in triples:
        yield (s,p,o,URIRef(graph))

In [3]:
if len(sg)==0:
    sg.parse("DMEAR_ser.rdf")
S = serialization.Serialization(sg, "EntityAttributeRecord")

In [4]:
file_dtypes = {'ModelDomain': str,
 'Model': str,
 'ModelType': str,
 'Class': str,
 'ClassDescription': str,
 'Attribute': str,
 'AttributeDescription': str,
 'DataType': str,
 'Context': str,
 'SubContext': str,
 'Nulls': str,
 'Is PK': str,
 'Relationship': str,
 'FromClass': str,
 'FromAttribute': str,
 'FromCardinality': str,
 'ToClass': str,
 'ToAttribute': str,
 'ToCardinality': str}

In [5]:
def get_field(v):
    if isinstance(v,str):
        return html.escape(str(v))
    elif isinstance(v,(int, float)):
        if pd.isnull(v):
            return None
        else:
            return v
    elif v is None or isinstance(v,pd.Null):
        return None

In [6]:
# Optional Delete data hosted in master and discourse database graphs
do="Yes"
do="No"

if do == "Yes":
    ds.update("""CLEAR GRAPH <http://master>""")
    ds.update("""CLEAR GRAPH <http://discourse>""")



In [7]:
#ds = Dataset(store=jena, default_union=True, default_graph_base="http://base.raw")
#master_g = ds.graph(URIRef("http://master"))
#discourse_g = ds.graph(URIRef("http://discourse"))


In [8]:
discourse_details_sparql = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX disco: <http://www.semanticweb.org/tomk/ontologies/2022/11/discourse#> 
PREFIX dcterms: <http://purl.org/dc/terms/> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 

select ?g ?discourse ?name ?title ?description ?created ?hash (COUNT(?declaration) as ?declarations) (COUNT(distinct ?psubject) as ?entities)

WHERE {
  GRAPH ?g
  {
      ?discourse a disco:Discourse.
      ?discourse dcterms:title ?title.
      ?discourse rdfs:label ?name.
      ?discourse dcterms:description ?description.
      ?discourse dcterms:created ?created .
      ?discourse disco:DiscourseHash ?hash .
      ?discourse disco:DiscourseContains+ ?declaration .
      ?declaration a disco:Declaration .
      ?posit_type rdfs:subPropertyOf* disco:Posits .
      ?declaration disco:Asserts ?assertion.
      ?assertion disco:Subject ?psubject.
  }
}
GROUP BY ?g ?discourse ?name ?title ?description ?created ?hash
"""

qr = ds.query(discourse_details_sparql)
discourse_df = pd.DataFrame(qr, columns=[v.n3()[1:] for v in qr.vars]).sort_values(by="created")
l_hashes = set(discourse_df['hash'].apply(lambda x : x.n3()[1:-1]))


In [9]:
l_hashes

{'3fb2b13976ae-9df5-8dfa-8fa797db2176',
 '942c61ee63bd-0f2b-481f-8fb4ae295b15',
 'b6ba48396968-61f8-ead7-a561d3e0b7ce'}

In [10]:
# Here master_g is a master graph sourced from Fuseki

model_data = pd.read_csv(r"sample/inventory_v1.csv", index_col="Sequence", dtype=file_dtypes)
d_p = meta_data_package_template( {"created" : "2023-04-10", 
                                   "creator" : "tomk", 
                                   "description" : "Sample ERD model version 1", 
                                   "title" : "inventory", 
                                   "modified" : "2023-04-10"})

rows = [dict({rk:get_field(rv) for rk, rv in r.items()}) for i,r in model_data.iterrows()]
mq,me_m = loader.get_triples(sg, "EntityAttributeRecord", rows, master_g, [])
p,de,di,h = loader.generate_discourse("Inventory ERD v1.0", mq, d_p)

if h in l_hashes:
    print("This data is already loaded! Aborting")
else:
    ds.addN(triples_to_quads(mq, URIRef("http://master")))
    ds.addN(triples_to_quads(p, URIRef("http://discourse")))
    ds.addN(triples_to_quads(de, URIRef("http://discourse")))
    ds.addN(triples_to_quads(di, URIRef("http://discourse")))


0:00:01.106535 for 9
This data is already loaded! Aborting


In [11]:
# Test duplicate data hash
# Here master_g is a master graph sourced from Fuseki
mq,me_m = loader.get_triples(sg, "EntityAttributeRecord", rows, master_g, [])
p,de,di,h = loader.generate_discourse("Inventory ERD v1.0", mq, d_p)


0:00:00.028728 for 9


In [12]:
model_data = pd.read_csv(r"sample/inventory_v2.csv", index_col="Sequence")
d_p = meta_data_package_template( {"created" : "2023-04-11", 
                                   "creator" : "tomk", 
                                   "description" : "Sample ERD model version 2", 
                                   "title" : "inventory ERD", 
                                   "modified" : "2023-04-11"})

rows = [dict({rk:get_field(rv) for rk, rv in r.items()}) for i,r in model_data.iterrows()]

# Here master_g is a master graph sourced from Fuseki
mq,me_m = loader.get_triples(sg, "EntityAttributeRecord", rows, master_g, [])
p,de,di,h = loader.generate_discourse("Inventory ERD v2.0", mq, d_p)

if h in l_hashes:
    print("This data is already loaded! Aborting")
else:
    ds.addN(triples_to_quads(mq, URIRef("http://master")))
    ds.addN(triples_to_quads(p, URIRef("http://discourse")))
    ds.addN(triples_to_quads(de, URIRef("http://discourse")))
    ds.addN(triples_to_quads(di, URIRef("http://discourse")))


0:00:00.043883 for 18
This data is already loaded! Aborting


In [13]:
dict(model_data.dtypes)

{'ModelDomain': dtype('O'),
 'Model': dtype('O'),
 'ModelType': dtype('O'),
 'Class': dtype('O'),
 'ClassDescription': dtype('O'),
 'Attribute': dtype('O'),
 'AttributeDescription': dtype('O'),
 'DataType': dtype('O'),
 'Context': dtype('float64'),
 'SubContext': dtype('float64'),
 'Nulls': dtype('O'),
 'Is PK': dtype('O'),
 'Relationship': dtype('O'),
 'FromClass': dtype('O'),
 'FromAttribute': dtype('O'),
 'FromCardinality': dtype('O'),
 'ToClass': dtype('O'),
 'ToAttribute': dtype('O'),
 'ToCardinality': dtype('O')}

In [14]:
model_data = pd.read_csv(r"sample/inventory_v3.csv", index_col="Sequence")
d_p = meta_data_package_template( {"created" : "2023-04-14", 
                                   "creator" : "tomk", 
                                   "description" : "Sample ERD model version 3", 
                                   "title" : "inventory ERD", 
                                   "modified" : "2023-04-14"})

rows = [dict({rk:get_field(rv) for rk, rv in r.items()}) for i,r in model_data.iterrows()]

# Here master_g is a master graph sourced from Fuseki
mq,me_m = loader.get_triples(sg, "EntityAttributeRecord", rows, master_g, [])
p,de,di,h = loader.generate_discourse("Inventory ERD v3.0", mq, d_p)

if h in l_hashes:
    print("This data is already loaded! Aborting")
else:
    ds.addN(triples_to_quads(mq, URIRef("http://master")))
    ds.addN(triples_to_quads(p, URIRef("http://discourse")))
    ds.addN(triples_to_quads(de, URIRef("http://discourse")))
    ds.addN(triples_to_quads(di, URIRef("http://discourse")))


0:00:00.094132 for 44
This data is already loaded! Aborting


In [15]:
# Step 1 - Tell me what discourses are available in the system

In [16]:
discourse_details_sparql = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX disco: <http://www.semanticweb.org/tomk/ontologies/2022/11/discourse#> 
PREFIX dcterms: <http://purl.org/dc/terms/> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 

select ?g ?discourse ?name ?title ?description ?created ?hash (COUNT(?declaration) as ?declarations) (COUNT(distinct ?psubject) as ?entities)

WHERE {
  GRAPH ?g
  {
      ?discourse a disco:Discourse.
      ?discourse dcterms:title ?title.
      ?discourse rdfs:label ?name.
      ?discourse dcterms:description ?description.
      ?discourse dcterms:created ?created .
      ?discourse disco:DiscourseHash ?hash .
      ?discourse disco:DiscourseContains+ ?declaration .
      ?declaration a disco:Declaration .
      ?posit_type rdfs:subPropertyOf* disco:Posits .
      ?declaration disco:Asserts ?assertion.
      ?assertion disco:Subject ?psubject.
  }
}
GROUP BY ?g ?discourse ?name ?title ?description ?created ?hash
"""

qr = ds.query(discourse_details_sparql)

In [17]:
discourse_df = pd.DataFrame(qr, columns=[v.n3()[1:] for v in qr.vars]).sort_values(by="created")
discourse_df

,g,discourse,name,title,description,created,hash,declarations,entities
0,http://discourse,http://www.semanticweb.org/tomk/ontologies/202...,discourse_Inventory ERD v1.0,inventory,Sample ERD model version 1,2023-04-10,b6ba48396968-61f8-ead7-a561d3e0b7ce,75,14
2,http://discourse,http://www.semanticweb.org/tomk/ontologies/202...,discourse_Inventory ERD v2.0,inventory ERD,Sample ERD model version 2,2023-04-11,942c61ee63bd-0f2b-481f-8fb4ae295b15,143,27
1,http://discourse,http://www.semanticweb.org/tomk/ontologies/202...,discourse_Inventory ERD v3.0,inventory ERD,Sample ERD model version 3,2023-04-14,3fb2b13976ae-9df5-8dfa-8fa797db2176,309,59


https://docs.data.world/tutorials/sparql/list-of-sparql-filter-functions.html

In [18]:
discourses = list(discourse_df['discourse'])
discourses

[rdflib.term.URIRef('http://www.semanticweb.org/tomk/ontologies/2022/11/faef05d7ca85497990ac538c1d026abb'),
 rdflib.term.URIRef('http://www.semanticweb.org/tomk/ontologies/2022/11/3db385b0618b4f68b8ad7c7c3d5197fa'),
 rdflib.term.URIRef('http://www.semanticweb.org/tomk/ontologies/2022/11/d25c5e4e5a474081b88a6809164a8415')]

In [19]:
discourses[0].n3()

'<http://www.semanticweb.org/tomk/ontologies/2022/11/faef05d7ca85497990ac538c1d026abb>'

In [20]:
get_discourse_posits_sparql = """

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX disco: <http://www.semanticweb.org/tomk/ontologies/2022/11/discourse#> 
PREFIX dcterms: <http://purl.org/dc/terms/> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 


select ?g ?discourse ?a ?r ?p
WHERE {
  GRAPH ?g
  {
      ?discourse a disco:Discourse.
      ?discourse disco:DiscourseContains+ ?declaration .
      ?declaration a disco:Declaration .
        OPTIONAL { ?declaration disco:Asserts ?a. 
                   }
        OPTIONAL { ?declaration disco:Refutes ?r. 
                   }
        OPTIONAL { ?declaration disco:Posits ?p. 
        }
      
  }
}

"""

qr = ds.query(get_discourse_posits_sparql)

pd.DataFrame(qr, columns=["g","d","a","r","p"]).groupby(["g","d"]).agg(set).iloc[2]['a']


{rdflib.term.URIRef('http://www.semanticweb.org/tomk/ontologies/2022/11/045be5162e8c-a508-68b2-6a6cb4a6c4e1'),
 rdflib.term.URIRef('http://www.semanticweb.org/tomk/ontologies/2022/11/054272d95a1d-1dd4-d6f2-2336037e764c'),
 rdflib.term.URIRef('http://www.semanticweb.org/tomk/ontologies/2022/11/057113d8fa05-21e5-536c-e265adaf6add'),
 rdflib.term.URIRef('http://www.semanticweb.org/tomk/ontologies/2022/11/08cd7306032f-8417-dc21-47e360158654'),
 rdflib.term.URIRef('http://www.semanticweb.org/tomk/ontologies/2022/11/097ef856a7c1-31b3-88ce-4dc68f987d5d'),
 rdflib.term.URIRef('http://www.semanticweb.org/tomk/ontologies/2022/11/0e863c0f9d1a-ff87-19d5-114e5259f159'),
 rdflib.term.URIRef('http://www.semanticweb.org/tomk/ontologies/2022/11/0eaac7debb2d-4f3d-c799-7f6d2dbb82e2'),
 rdflib.term.URIRef('http://www.semanticweb.org/tomk/ontologies/2022/11/121fd33fff16-1bfe-3541-1b2287efb136'),
 rdflib.term.URIRef('http://www.semanticweb.org/tomk/ontologies/2022/11/13927cbbd2e7-2592-27c0-c9a55d1f816f'),
 

In [21]:
# Given two discourse urls, d1, d2, pull all the discourse-data associated 
# - including the reified triples they reference


get_posits_by_discourse_sparql = """

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX disco: <http://www.semanticweb.org/tomk/ontologies/2022/11/discourse#> 
PREFIX dcterms: <http://purl.org/dc/terms/> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 

select ?g ?discourse ?assertion ?refutation ?posit ?s ?p ?o
WHERE {
  GRAPH ?g
  {
      ?discourse a disco:Discourse.
      ?discourse disco:DiscourseContains+ ?declaration .
      ?declaration a disco:Declaration .
        OPTIONAL { ?declaration disco:Asserts ?assertion. 
                   ?assertion disco:Subject ?s.
                   ?assertion disco:Predicate ?p.
                   ?assertion disco:Object ?o.
                   }
        OPTIONAL { ?declaration disco:Refutes ?refutation. 
                   ?refutation disco:Subject ?s.
                   ?refutation disco:Predicate ?p.
                   ?refutation disco:Object ?o.
                   }
        OPTIONAL { ?declaration disco:Posits ?posit. 
                   ?posit disco:Subject ?s.
                   ?posit disco:Predicate ?p.
                   ?posit disco:Object ?o.
        }
      FILTER (?discourse IN ( %%d1%%, %%d2%% )) .
      
  }
}

""".replace("%%d1%%",discourses[0].n3()).replace("%%d2%%", discourses[2].n3())

qr = ds.query(get_posits_by_discourse_sparql)

In [22]:
discourse_contents = pd.DataFrame(qr, columns=[v.n3()[1:] for v in qr.vars]).sort_values(by="discourse")

In [23]:
discourse_contents['triple'] = discourse_contents.apply(lambda x : tuple([x['s'], x['p'], x['o']]), axis=1)

In [24]:
d_set_d = discourse_contents.groupby(['discourse'])['triple'].agg(set).to_dict()


#d_set_d = discourse_contents.groupby(['discourse']).agg(set).to_dict()


In [25]:
d_set_d

{rdflib.term.URIRef('http://www.semanticweb.org/tomk/ontologies/2022/11/d25c5e4e5a474081b88a6809164a8415'): {(rdflib.term.URIRef('http://www.semanticweb.org/tomk/ontologies/2022/11/0d253b14811741249cfeb315cb7943ec'),
   rdflib.term.URIRef('http://www.semanticweb.org/tomk/ontologies/2022/11/serialization#UniqueIdentifier'),
   rdflib.term.Literal('Inventory.Simple ERD Model - Inventory.OrderItem.price_per_unit')),
  (rdflib.term.URIRef('http://www.semanticweb.org/tomk/ontologies/2022/11/0d253b14811741249cfeb315cb7943ec'),
   rdflib.term.URIRef('http://www.semanticweb.org/tomk/ontologies/2022/9/datamodel#DataType'),
   rdflib.term.Literal('float')),
  (rdflib.term.URIRef('http://www.semanticweb.org/tomk/ontologies/2022/11/0d253b14811741249cfeb315cb7943ec'),
   rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
   rdflib.term.URIRef('http://www.semanticweb.org/tomk/ontologies/2022/9/datamodel#Attribute')),
  (rdflib.term.URIRef('http://www.semanticweb.org/tomk/ontologi

In [26]:
key_l = list(d_set_d.keys())

In [27]:
def diffset(S1, S2):
    # Given two input sets, s1 and s2, return the Left difference, Intersection and Right difference between them
    L = S1.difference(S2)
    I = S1.intersection(S2)
    R = S2.difference(S1)
    return L,I,R

In [28]:
import hashlib

In [29]:
print("(intersection, left_diff, right_diff)")
len(d_set_d[key_l[0]].intersection(d_set_d[key_l[1]])), len(d_set_d[key_l[0]].difference(d_set_d[key_l[1]])), len(d_set_d[key_l[1]].difference(d_set_d[key_l[0]]))
s1,s2 = d_set_d[key_l[0]], d_set_d[key_l[1]]
diffs = diffset(s1,s2)
[len(d) for d in diffs], len(s1), len(s2)


(intersection, left_diff, right_diff)


([234, 75, 0], 309, 75)

In [30]:
list(triples_to_quads(diffs[0], URIRef("http://temp")))

[(rdflib.term.URIRef('http://www.semanticweb.org/tomk/ontologies/2022/11/c66ffaf64c4546afa212aa53728be6fb'),
  rdflib.term.URIRef('http://www.semanticweb.org/tomk/ontologies/2022/9/datamodel#DataType'),
  rdflib.term.Literal('float'),
  rdflib.term.URIRef('http://temp')),
 (rdflib.term.URIRef('http://www.semanticweb.org/tomk/ontologies/2022/11/f4c4608a471144bd8b28744c472a0eb8'),
  rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'),
  rdflib.term.Literal('customer_address_line_1'),
  rdflib.term.URIRef('http://temp')),
 (rdflib.term.URIRef('http://www.semanticweb.org/tomk/ontologies/2022/11/6382c75e14af4c31be5e272b283991e1'),
  rdflib.term.URIRef('http://www.semanticweb.org/tomk/ontologies/2022/9/datamodel#Contains'),
  rdflib.term.URIRef('http://www.semanticweb.org/tomk/ontologies/2022/11/484dd6c13cc04e15bde0c8eda56caae6'),
  rdflib.term.URIRef('http://temp')),
 (rdflib.term.URIRef('http://www.semanticweb.org/tomk/ontologies/2022/11/acd08011bc0d49bdb10b4b27bde770b2'),
  r

In [31]:
test_g = Graph()
test_g.addN(triples_to_quads(diffs[0], URIRef("http://temp")))
list(test_g.triples((None, None, None)))

[]

In [32]:
def _addN(graph, triples):
    for t in triples:
        graph.add(t)
    return graph

# Feed diffset split sets of SPO triples
def visualise_ERD_diffset(L, I, R):
#    L = original only
#    I = both
#    R = new only
    lir_graphs = _addN(Graph(), L), _addN(Graph(), I), _addN(Graph(), R)
    # Extract identifiers for all main model components
    lir_entities = [[s for s,p,o in gg.triples((None, URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), URIRef('http://www.semanticweb.org/tomk/ontologies/2022/9/datamodel#Class')))] for gg in lir_graphs]
    lir_attributes = [[s for s,p,o in gg.triples((None, URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), URIRef('http://www.semanticweb.org/tomk/ontologies/2022/9/datamodel#Attribute')))] for gg in lir_graphs]
    lir_rels  = [[s for s,p,o in gg.triples((None, URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), URIRef('http://www.semanticweb.org/tomk/ontologies/2022/9/datamodel#Relationship')))] for gg in lir_graphs]
    lir_contexts  = [[s for s,p,o in gg.triples((None, URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), URIRef('http://www.semanticweb.org/tomk/ontologies/2022/9/datamodel#Context')))] for gg in lir_graphs]
    lir_models  = [[s for s,p,o in gg.triples((None, URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), URIRef('http://www.semanticweb.org/tomk/ontologies/2022/9/datamodel#Model')))] for gg in lir_graphs]
    lir_domains  = [[s for s,p,o in gg.triples((None, URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), URIRef('http://www.semanticweb.org/tomk/ontologies/2022/9/datamodel#ModelDomain')))] for gg in lir_graphs]
    
    
    
    print(lir_entities, "\n\n", lir_models)
    return l_graph
    

def build_entity_table(entity_url, )    



SyntaxError: invalid syntax (<ipython-input-32-1570f7b6c5f5>, line 26)

In [ ]:
def get_atom_set(triple_set, indx):
    subjects=set()
    for t in triple_set:
        subjects.add(t[indx])
    return subjects

def get_subjects(triple_set):
    return get_atom_set(triple_set, 0)

def get_predicates(triple_set):
    return get_atom_set(triple_set, 1)

def get_objects(triple_set):
    return get_atom_set(triple_set, 2)

def get_type_atoms(triple_set, indx):
    typed=set()
    for t in triple_set:
        if t[1]==URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type"):
            typed.add(t[indx])
    return typed

def get_types(triple_set):
    return get_type_atoms(triple_set, 2)

def get_typed_objects(triple_set):
    return get_type_atoms(triple_set, 0)


In [ ]:
L,I,R = diffset(s1, s2)
subset = get_subjects(I)
predset = get_predicates(I) 
obset = get_objects(I)
typeset = get_typed_objects(I)
types = get_types(I)

typeset.symmetric_difference(obset).symmetric_difference(types)
#list(ds.triples((URIRef('http://www.semanticweb.org/tomk/ontologies/2022/11/b3b0fd89605e4d04a3f698d974d3579f'), None, None)))

In [ ]:
[len(q) for q in [L,I,R]]

In [ ]:
#discourse_contents[discourse_contents['p']==][['s','p','o']]
discourse_contents[['s','p','o']].groupby("p").agg(set).loc[URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type')]['o']



In [ ]:
discourses

In [ ]:
visualize(t2rdflibg( discourse_contents[discourse_contents['discourse']==discourses[1]][['s','p','o']].values))